In [1]:
# Importing Libraries 

import pandas as pd
import numpy as np
import math
from pyomo.environ import *

In [2]:
from __future__ import division
from pyomo.opt import SolverFactory

In [3]:
# Reading the data from Excel workbook
 
Staff = pd.read_excel("D:\Renu\My Personal Docs\Education Program\CaseStudyPyomo\Staffing+Data.xlsx", sheet_name='StaffAvailability')
Cost = pd.read_excel("D:\Renu\My Personal Docs\Education Program\CaseStudyPyomo\Staffing+Data.xlsx", sheet_name='Cost')
ServiceRate = pd.read_excel("D:\Renu\My Personal Docs\Education Program\CaseStudyPyomo\Staffing+Data.xlsx", sheet_name='ServiceRate')
InsDemand = pd.read_excel("D:\Renu\My Personal Docs\Education Program\CaseStudyPyomo\Staffing+Data.xlsx", sheet_name='DemandData')

## Data pre-processing 

In [4]:
InsDemand.head()

State Month  Demand
0     A   Jan    5240
1     A   Feb    4878
2     A   Mar    5942
3     A   Apr    2297
4     A   May    1992

In [5]:
Cost.head()

State Month  AnnualSalary  MonthlySalary  UnitOutSourceCost
0     A   Jan         60000         5000.0                180
1     A   Feb         60000         5000.0                180
2     A   Mar         60000         5000.0                180
3     A   Apr         60000         5000.0                180
4     A   May         60000         5000.0                180

In [6]:
ServiceRate.head()

MgAppServedPerMonth
0                   40

In [7]:
Staff.head()

State Month    LB    UB  StaffAvPer
0     A   Jan  0.70  0.90        0.81
1     A   Feb  0.65  0.85        0.76
2     A   Mar  0.70  0.80        0.75
3     A   Apr  0.75  0.85        0.80
4     A   May  0.70  0.85        0.78

In [8]:
# Checking unique State
Cost['State'].unique()

array(['A', 'B', 'C'], dtype=object)

In [9]:
# checking unique months
Cost['Month'].unique()

array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'], dtype=object)

## Step 1 : Define Indexes & Parameter

In [10]:
# Creating structures for Indexes and Parameter
# Demand of Insurance Per State & Month
Demand=InsDemand.set_index(['State','Month'])['Demand'].to_dict()
Demand

{('A', 'Jan'): 5240,
 ('A', 'Feb'): 4878,
 ('A', 'Mar'): 5942,
 ('A', 'Apr'): 2297,
 ('A', 'May'): 1992,
 ('A', 'Jun'): 2275,
 ('A', 'Jul'): 5334,
 ('A', 'Aug'): 3371,
 ('A', 'Sep'): 3759,
 ('A', 'Oct'): 3529,
 ('A', 'Nov'): 4284,
 ('A', 'Dec'): 5183,
 ('B', 'Jan'): 4927,
 ('B', 'Feb'): 2628,
 ('B', 'Mar'): 2974,
 ('B', 'Apr'): 2338,
 ('B', 'May'): 4020,
 ('B', 'Jun'): 3147,
 ('B', 'Jul'): 4271,
 ('B', 'Aug'): 2620,
 ('B', 'Sep'): 4517,
 ('B', 'Oct'): 4155,
 ('B', 'Nov'): 3137,
 ('B', 'Dec'): 4227,
 ('C', 'Jan'): 1162,
 ('C', 'Feb'): 1967,
 ('C', 'Mar'): 1898,
 ('C', 'Apr'): 2261,
 ('C', 'May'): 2030,
 ('C', 'Jun'): 1642,
 ('C', 'Jul'): 2489,
 ('C', 'Aug'): 2496,
 ('C', 'Sep'): 922,
 ('C', 'Oct'): 2421,
 ('C', 'Nov'): 963,
 ('C', 'Dec'): 1998}

In [11]:
# 2. FTE Salary
FTE_Salary = Cost.set_index(['State','Month'])['MonthlySalary'].to_dict()
FTE_Salary

{('A', 'Jan'): 5000.0,
 ('A', 'Feb'): 5000.0,
 ('A', 'Mar'): 5000.0,
 ('A', 'Apr'): 5000.0,
 ('A', 'May'): 5000.0,
 ('A', 'Jun'): 5000.0,
 ('A', 'Jul'): 5000.0,
 ('A', 'Aug'): 5000.0,
 ('A', 'Sep'): 5000.0,
 ('A', 'Oct'): 5000.0,
 ('A', 'Nov'): 5000.0,
 ('A', 'Dec'): 5000.0,
 ('B', 'Jan'): 4583.333333333333,
 ('B', 'Feb'): 4583.333333333333,
 ('B', 'Mar'): 4583.333333333333,
 ('B', 'Apr'): 4583.333333333333,
 ('B', 'May'): 4583.333333333333,
 ('B', 'Jun'): 4583.333333333333,
 ('B', 'Jul'): 4583.333333333333,
 ('B', 'Aug'): 4583.333333333333,
 ('B', 'Sep'): 4583.333333333333,
 ('B', 'Oct'): 4583.333333333333,
 ('B', 'Nov'): 4583.333333333333,
 ('B', 'Dec'): 4583.333333333333,
 ('C', 'Jan'): 4416.666666666667,
 ('C', 'Feb'): 4416.666666666667,
 ('C', 'Mar'): 4416.666666666667,
 ('C', 'Apr'): 4416.666666666667,
 ('C', 'May'): 4416.666666666667,
 ('C', 'Jun'): 4416.666666666667,
 ('C', 'Jul'): 4416.666666666667,
 ('C', 'Aug'): 4416.666666666667,
 ('C', 'Sep'): 4416.666666666667,
 ('C', 'Oc

In [12]:
# 3. Unit Outsource Cost
UnitOutSourceCost = Cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()
UnitOutSourceCost

{('A', 'Jan'): 180,
 ('A', 'Feb'): 180,
 ('A', 'Mar'): 180,
 ('A', 'Apr'): 180,
 ('A', 'May'): 180,
 ('A', 'Jun'): 180,
 ('A', 'Jul'): 180,
 ('A', 'Aug'): 180,
 ('A', 'Sep'): 180,
 ('A', 'Oct'): 180,
 ('A', 'Nov'): 180,
 ('A', 'Dec'): 180,
 ('B', 'Jan'): 150,
 ('B', 'Feb'): 150,
 ('B', 'Mar'): 150,
 ('B', 'Apr'): 150,
 ('B', 'May'): 150,
 ('B', 'Jun'): 150,
 ('B', 'Jul'): 150,
 ('B', 'Aug'): 150,
 ('B', 'Sep'): 150,
 ('B', 'Oct'): 150,
 ('B', 'Nov'): 150,
 ('B', 'Dec'): 150,
 ('C', 'Jan'): 160,
 ('C', 'Feb'): 160,
 ('C', 'Mar'): 160,
 ('C', 'Apr'): 160,
 ('C', 'May'): 160,
 ('C', 'Jun'): 160,
 ('C', 'Jul'): 160,
 ('C', 'Aug'): 160,
 ('C', 'Sep'): 160,
 ('C', 'Oct'): 160,
 ('C', 'Nov'): 160,
 ('C', 'Dec'): 160}

In [13]:
# 4. Number of Applications when Employee is working Full time i.e. 100% Utilization
FTE_AppServeRate = ServiceRate.iloc[0,0]
FTE_AppServeRate

40

In [14]:
# 5. Staff Availability for serving the Insurance Appllication
Staff_LB = Staff.set_index(['State','Month'])['LB'].to_dict()
Staff_UB = Staff.set_index(['State','Month'])['UB'].to_dict()
StaffAvg = Staff.set_index(['State','Month'])['StaffAvPer'].to_dict()

Staff_LB


{('A', 'Jan'): 0.7,
 ('A', 'Feb'): 0.65,
 ('A', 'Mar'): 0.7,
 ('A', 'Apr'): 0.75,
 ('A', 'May'): 0.7,
 ('A', 'Jun'): 0.65,
 ('A', 'Jul'): 0.6,
 ('A', 'Aug'): 0.65,
 ('A', 'Sep'): 0.7,
 ('A', 'Oct'): 0.65,
 ('A', 'Nov'): 0.6,
 ('A', 'Dec'): 0.6,
 ('B', 'Jan'): 0.7,
 ('B', 'Feb'): 0.65,
 ('B', 'Mar'): 0.7,
 ('B', 'Apr'): 0.75,
 ('B', 'May'): 0.7,
 ('B', 'Jun'): 0.65,
 ('B', 'Jul'): 0.6,
 ('B', 'Aug'): 0.65,
 ('B', 'Sep'): 0.7,
 ('B', 'Oct'): 0.65,
 ('B', 'Nov'): 0.6,
 ('B', 'Dec'): 0.6,
 ('C', 'Jan'): 0.7,
 ('C', 'Feb'): 0.65,
 ('C', 'Mar'): 0.7,
 ('C', 'Apr'): 0.75,
 ('C', 'May'): 0.7,
 ('C', 'Jun'): 0.65,
 ('C', 'Jul'): 0.6,
 ('C', 'Aug'): 0.65,
 ('C', 'Sep'): 0.7,
 ('C', 'Oct'): 0.65,
 ('C', 'Nov'): 0.6,
 ('C', 'Dec'): 0.6}

In [15]:
# 6. Index
BankLoc = Cost['State'].unique()
Month = Cost['Month'].unique()
BankLoc , Month

(array(['A', 'B', 'C'], dtype=object),
 array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
        'Oct', 'Nov', 'Dec'], dtype=object))

# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



**Note:** You can write the mathematical formulation directly in your report.

# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [16]:
# Creating a model instance
model = ConcreteModel()

In [17]:
# Creating Indexes for State & Month
# i = A, B, C
# j = Months Jan, Feb,....,Dec

model.i = Set(initialize=BankLoc.tolist(), doc='States')
model.j = Set(initialize=Month.tolist(), doc='Months')

In [18]:
# Define Parameters for Demand, FTE Salaries, Outsourcing Cost, Avg Staff Availability

model.demand = Param(model.i, model.j, initialize=Demand, doc='Demand')
model.salary = Param(model.i, model.j, initialize=FTE_Salary, doc='FTE_Salary')
model.cost = Param(model.i, model.j, initialize=UnitOutSourceCost, doc='UnitOutSourceCost')
model.staffavl = Param(model.i, model.j, initialize=StaffAvg, doc='StaffAverageAvl')

In [19]:
# Decision variables
model.x = Var(model.i, model.j, within=NonNegativeReals, doc='No of FTE')
model.y = Var(model.i, model.j, within=NonNegativeIntegers, doc='No of Outsourced App')

In [20]:
# Defining Constraints for Total Demand

def total_demand(m,i,j):
    return (m.x[i,j]*40*m.staffavl[i,j] + m.y[i,j] == m.demand[i,j])
            
model.total_demand = Constraint(model.i,model.j, rule=total_demand)      


In [21]:
# Parameters for restriction: Regulatory

# Regulatory Constraint for A
def Reg_A(model, i, j):
    return 0.30 * model.demand[i,j] # 30*total_demand

model.Reg_A = Param(model.i, model.j, initialize=Reg_A, doc='RegRest_A')

# Regulatory Constraint for B
def Reg_B(model, i, j):
    return 0.40 * model.demand[i,j] # 40*total_demand

model.Reg_B = Param(model.i, model.j, initialize=Reg_B, doc='RegRest_B')

In [22]:
# Regulatory Constraint
model.regulatoryCons = ConstraintList()
for i in model.i:
    for j in model.j:
        if i == 'A':
            model.regulatoryCons.add(expr = model.y[i,j] <= model.Reg_A[i,j])
        elif i == 'B':
            model.regulatoryCons.add(expr = model.y[i,j] <= model.Reg_B[i,j])

In [23]:
# Defining Objective function

# Minimize Staff Cost

def objective_rule(model):
    return sum(model.salary[i,j]*model.x[i,j] for i in model.i for j in model.j) +\
            sum(model.cost[i,j]*model.y[i,j] for i in model.i for j in model.j)


model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define Objective Function')

In [24]:
# Invoking the solver

result_cost = SolverFactory('glpk').solve(model)
result_cost.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17962336.4487699
  Upper bound: 17962336.4487699
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.1738758087158203
# ----------------------------------------------------------
#   Solution Information
# -------------

In [25]:
# Print the value of the objective function
optimized_cost = round(model.objective()/1000000,2)
optimized_cost

17.96

**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [26]:
model.pprint()

12 Set Declarations
    Reg_A_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   36 : {('A', 'Jan'), ('A', 'Feb'), ('A', 'Mar'), ('A', 'Apr'), ('A', 'May'), ('A', 'Jun'), ('A', 'Jul'), ('A', 'Aug'), ('A', 'Sep'), ('A', 'Oct'), ('A', 'Nov'), ('A', 'Dec'), ('B', 'Jan'), ('B', 'Feb'), ('B', 'Mar'), ('B', 'Apr'), ('B', 'May'), ('B', 'Jun'), ('B', 'Jul'), ('B', 'Aug'), ('B', 'Sep'), ('B', 'Oct'), ('B', 'Nov'), ('B', 'Dec'), ('C', 'Jan'), ('C', 'Feb'), ('C', 'Mar'), ('C', 'Apr'), ('C', 'May'), ('C', 'Jun'), ('C', 'Jul'), ('C', 'Aug'), ('C', 'Sep'), ('C', 'Oct'), ('C', 'Nov'), ('C', 'Dec')}
    Reg_B_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   36 : {('A', 'Jan'), ('A', 'Feb'), ('A', 'Mar'), ('A', 'Apr'), ('A', 'May'), ('A', 'Jun'), ('A', 'Jul'), ('A', 'Aug'), ('A', 'Sep'), ('A', 'Oct'), ('A', 'Nov'), ('A', 'Dec'), ('B', 'Jan'), ('B', 'Feb'), ('B'

In [27]:
# Creating dataframe for the results

final_result=[]

for i in model.i:
    for j in model.j:
        demand=model.demand[i,j]
        no_of_fte=round(model.x[i,j].value,2) 
        no_of_outsource=round(model.y[i,j].value,2)
        cost=round(no_of_fte*model.salary[i,j] + no_of_outsource*model.cost[i,j],2)
        no_of_FTE_processed_app = model.x[i,j].value*40*model.staffavl[i,j]
        per_outsource=round((no_of_outsource/demand)*100,2)
        per_fte=round((no_of_FTE_processed_app/demand)*100,2)
        avg_cost_per_app=round(cost/demand,2)
        final_result.append([i,j,demand,no_of_fte,no_of_outsource,per_fte,per_outsource,cost,avg_cost_per_app,no_of_FTE_processed_app])
        

df=pd.DataFrame(final_result,columns=['State','Month','Demand','No Of FTE','No of Outsourced Appl','Appl Percentage FTE',
                                      'Appl Percentage Outsource','Cost','Average Cost Per App','No Of FTE Processed Appl'])    
df.head(10)

State Month  Demand  No Of FTE  No of Outsourced Appl  Appl Percentage FTE  \
0     A   Jan    5240     161.73                    0.0                100.0   
1     A   Feb    4878     160.46                    0.0                100.0   
2     A   Mar    5942     198.07                    0.0                100.0   
3     A   Apr    2297      71.78                    0.0                100.0   
4     A   May    1992      63.85                    0.0                100.0   
5     A   Jun    2275      77.91                    0.0                100.0   
6     A   Jul    5334     137.28                 1600.0                 70.0   
7     A   Aug    3371     110.89                    0.0                100.0   
8     A   Sep    3759     116.02                    0.0                100.0   
9     A   Oct    3529     120.86                    0.0                100.0   

   Appl Percentage Outsource      Cost  Average Cost Per App  \
0                        0.0  808650.0                154.32   
1                        0.0  802300.0                164.47   
2                        0.0  990350.0                166.67   
3                        0.0  358900.0                156.25   
4                        0.0  319250.0                160.27   
5                        0.0  389550.0                171.23   
6                       30.0  974400.0                182.68   
7                        0.0  554450.0                164.48   
8                        0.0  580100.0                154.32   
9                        0.0  604300.0                171.24   

   No Of FTE Processed Appl  
0                    5240.0  
1                    4878.0  
2                    5942.0  
3                    2297.0  
4                    1992.0  
5                    2275.0  
6                    3734.0  
7                    3371.0  
8                    3759.0  
9                    3529.0

In [28]:
# Writing the results in to an Excel sheet
from openpyxl import load_workbook
def append_to_excel(fpath, df, sheet_name,md):
    with pd.ExcelWriter(fpath, mode=md, engine='openpyxl') as f:
        df.to_excel(f, sheet_name=sheet_name)

append_to_excel('D:/StaffOptimizationResults.xlsx', df, 'Average Availability','w')


# Question 3

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 3.1 Worst case analysis 

- 3.1.1 What is the optimal number of staff members for the worst case? 

- 3.1.2 What is the percentage of outsourcing for the worst case? 

- 3.1.3 What is the average cost per application for the worst case?


#### 3.2 Best case analysis 

- 3.2.1 What is the optimal number of staff members for the best case? 

- 3.2.2 What is the percentage of outsourcing for the best case? 

- 3.2.3 What is the average cost per application for the best case?


#### Expected output:

For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 3.1 Worst Case Analysis 

#### 3.1.1 Optimal number of staff members

In [29]:
# Creating a model instance
modelworst = ConcreteModel()

In [30]:
# Creating Indexes for State & Month
# i = A, B, C
# j = Months Jan, Feb,....,Dec

modelworst.i = Set(initialize=BankLoc.tolist(), doc='States')
modelworst.j = Set(initialize=Month.tolist(), doc='Months')

In [31]:
# Define Parameters for Demand, FTE Salaries, Outsourcing Cost, Avg Staff Availability

modelworst.demand = Param(modelworst.i, modelworst.j, initialize=Demand, doc='Demand')
modelworst.salary = Param(modelworst.i, modelworst.j, initialize=FTE_Salary, doc='FTE_Salary')
modelworst.cost = Param(modelworst.i, modelworst.j, initialize=UnitOutSourceCost, doc='UnitOutSourceCost')
modelworst.staffavl = Param(modelworst.i, modelworst.j, initialize=Staff_LB, doc='StaffLB')

In [32]:
# Decision variables
modelworst.x = Var(modelworst.i, modelworst.j, within=NonNegativeReals, doc='No of FTE')
modelworst.y = Var(modelworst.i, modelworst.j, within=NonNegativeIntegers, doc='No of Outsourced App')

In [33]:
# Defining Constraints for Total Demand
modelworst.total_demand = Constraint(modelworst.i,modelworst.j, rule=total_demand)      


In [34]:
# Parameters for restriction: Regulatory
modelworst.Reg_A = Param(modelworst.i, modelworst.j, initialize=Reg_A, doc='RegRest_A')
modelworst.Reg_B = Param(modelworst.i, modelworst.j, initialize=Reg_B, doc='RegRest_B')

In [35]:
# Regulatory Constraint
modelworst.regulatoryCons = ConstraintList()
for i in modelworst.i:
    for j in modelworst.j:
        if i == 'A':
            modelworst.regulatoryCons.add(expr = modelworst.y[i,j] <= modelworst.Reg_A[i,j])
        elif i == 'B':
            modelworst.regulatoryCons.add(expr = modelworst.y[i,j] <= modelworst.Reg_B[i,j])

In [36]:
# Defining Objective function

# Minimize Staff Cost

def objective_rule(model):
    return sum(model.salary[i,j]*model.x[i,j] for i in model.i for j in model.j) +\
            sum(model.cost[i,j]*model.y[i,j] for i in model.i for j in model.j)


modelworst.objective = Objective(rule=objective_rule, sense=minimize, doc='Define Objective Function')

In [37]:
# Invoking the solver
result = SolverFactory('glpk').solve(modelworst)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 19599482.5167888
  Upper bound: 19599482.5167888
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.12444472312927246
# ----------------------------------------------------------
#   Solution Information
# ------------

In [38]:
# Print the value of the objective function
optimized_cost = round(modelworst.objective()/1000000,2)
optimized_cost

19.6

**`Checkpoint 2:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.

In [39]:
# Creating dataframe for the results

final_result_worst=[]

for i in modelworst.i:
    for j in modelworst.j:
        demand=modelworst.demand[i,j]
        no_of_fte=round(modelworst.x[i,j].value,2) 
        no_of_outsource=round(modelworst.y[i,j].value,2)
        cost=round(no_of_fte*modelworst.salary[i,j] + no_of_outsource*modelworst.cost[i,j],2)
        no_of_FTE_processed_app = modelworst.x[i,j].value*40*modelworst.staffavl[i,j]
        per_outsource=round((no_of_outsource/demand)*100,2)
        per_fte=round((no_of_FTE_processed_app/demand)*100,2)
        avg_cost_per_app=round(cost/demand,2)
        final_result_worst.append([i,j,demand,no_of_fte,no_of_outsource,per_fte,per_outsource,cost,avg_cost_per_app,no_of_FTE_processed_app])
        

dfworst=pd.DataFrame(final_result_worst,columns=['State','Month','Demand','No Of FTE','No of Outsourced Appl','Appl Percentage FTE',
                                      'Appl Percentage Outsource','Cost','Average Cost Per App','No Of FTE Processed Appl'])    
dfworst.head(10)

State Month  Demand  No Of FTE  No of Outsourced Appl  Appl Percentage FTE  \
0     A   Jan    5240     187.14                    0.0               100.00   
1     A   Feb    4878     131.35                 1463.0                70.01   
2     A   Mar    5942     212.21                    0.0               100.00   
3     A   Apr    2297      76.57                    0.0               100.00   
4     A   May    1992      71.14                    0.0               100.00   
5     A   Jun    2275      61.27                  682.0                70.02   
6     A   Jul    5334     155.58                 1600.0                70.00   
7     A   Aug    3371      90.77                 1011.0                70.01   
8     A   Sep    3759     134.25                    0.0               100.00   
9     A   Oct    3529      95.04                 1058.0                70.02   

   Appl Percentage Outsource       Cost  Average Cost Per App  \
0                       0.00   935700.0                178.57   
1                      29.99   920090.0                188.62   
2                       0.00  1061050.0                178.57   
3                       0.00   382850.0                166.67   
4                       0.00   355700.0                178.56   
5                      29.98   429110.0                188.62   
6                      30.00  1065900.0                199.83   
7                      29.99   635830.0                188.62   
8                       0.00   671250.0                178.57   
9                      29.98   665640.0                188.62   

   No Of FTE Processed Appl  
0                    5240.0  
1                    3415.0  
2                    5942.0  
3                    2297.0  
4                    1992.0  
5                    1593.0  
6                    3734.0  
7                    2360.0  
8                    3759.0  
9                    2471.0

In [40]:
# sending data to exce;
append_to_excel('D:/StaffOptimizationResults.xlsx', dfworst, 'Lower Availability','a')

#### 3.1.2 Percentage of outsourced applications 

In [41]:
# write your code here
outsourcedper = round((dfworst['No of Outsourced Appl'].sum() / dfworst['Demand'].sum() ) * 100,2)
print("% of Outsourced Applications are : ", outsourcedper)

% of Outsourced Applications are :  35.14


#### 3.1.3 Average cost per application

In [42]:
# write your code here
avgcost= round(dfworst['Cost'].sum()/dfworst['Demand'].sum()  , 2)
print("Average Cost per Application :", avgcost)

Average Cost per Application : 173.0


### 3.2  Best Case Analysis 

#### 3.2.1 Optimal number of staff members


In [43]:
# Creating a model instance
modelbest = ConcreteModel()

In [44]:
# Creating Indexes for State & Month
# i = A, B, C
# j = Months Jan, Feb,....,Dec

modelbest.i = Set(initialize=BankLoc.tolist(), doc='States')
modelbest.j = Set(initialize=Month.tolist(), doc='Months')

In [45]:
# Define Parameters for Demand, FTE Salaries, Outsourcing Cost, Avg Staff Availability

modelbest.demand = Param(modelbest.i, modelbest.j, initialize=Demand, doc='Demand')
modelbest.salary = Param(modelbest.i, modelbest.j, initialize=FTE_Salary, doc='FTE_Salary')
modelbest.cost = Param(modelbest.i, modelbest.j, initialize=UnitOutSourceCost, doc='UnitOutSourceCost')
modelbest.staffavl = Param(modelbest.i, modelbest.j, initialize=Staff_UB, doc='StaffLB')

In [46]:
# Decision variables
modelbest.x = Var(modelbest.i, modelbest.j, within=NonNegativeReals, doc='No of FTE')
modelbest.y = Var(modelbest.i, modelbest.j, within=NonNegativeIntegers, doc='No of Outsourced App')

In [47]:
# Defining Constraints for Total Demand
modelbest.total_demand = Constraint(modelbest.i,modelbest.j, rule=total_demand)      


In [48]:
# Parameters for restriction: Regulatory , calling already defined functions

modelbest.Reg_A = Param(modelbest.i, modelbest.j, initialize=Reg_A, doc='RegRest_A')
modelbest.Reg_B = Param(modelbest.i, modelbest.j, initialize=Reg_B, doc='RegRest_B')

In [49]:
# Regulatory Constraint
modelbest.regulatoryCons = ConstraintList()
for i in modelbest.i:
    for j in modelbest.j:
        if i == 'A':
            modelbest.regulatoryCons.add(expr = modelbest.y[i,j] <= modelbest.Reg_A[i,j])
        elif i == 'B':
            modelbest.regulatoryCons.add(expr = modelbest.y[i,j] <= modelbest.Reg_B[i,j])

In [50]:
# Defining Objective function
# Minimize Staff Cost
modelbest.objective = Objective(rule=objective_rule, sense=minimize, doc='Define Objective Function')

In [51]:
# Invoking the solver
resultbest = SolverFactory('glpk').solve(modelbest)
resultbest.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16527535.6379357
  Upper bound: 16527535.6379357
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.13086342811584473
# ----------------------------------------------------------
#   Solution Information
# ------------

In [52]:
# Print the value of the objective function
optimized_cost = round(modelbest.objective()/1000000,2)
optimized_cost

16.53

**`Checkpoint 3:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [53]:
# Creating dataframe for the results

final_result_best=[]

for i in modelbest.i:
    for j in modelbest.j:
        demand=modelbest.demand[i,j]
        no_of_fte=round(modelbest.x[i,j].value,2) 
        no_of_outsource=round(modelbest.y[i,j].value,2)
        cost=round(no_of_fte*modelbest.salary[i,j] + no_of_outsource*modelbest.cost[i,j],2)
        no_of_FTE_processed_app = modelbest.x[i,j].value*40*modelbest.staffavl[i,j]
        per_outsource=round((no_of_outsource/demand)*100,2)
        per_fte=round((no_of_FTE_processed_app/demand)*100,2)
        avg_cost_per_app=round(cost/demand,2)
        final_result_best.append([i,j,demand,no_of_fte,no_of_outsource,per_fte,per_outsource,cost,avg_cost_per_app,no_of_FTE_processed_app])
        

dfbest=pd.DataFrame(final_result_best,columns=['State','Month','Demand','No Of FTE','No of Outsourced Appl','Appl Percentage FTE',
                                      'Appl Percentage Outsource','Cost','Average Cost Per App','No Of FTE Processed Appl'])    
dfbest.head(10)

State Month  Demand  No Of FTE  No of Outsourced Appl  Appl Percentage FTE  \
0     A   Jan    5240     145.56                    0.0                100.0   
1     A   Feb    4878     143.47                    0.0                100.0   
2     A   Mar    5942     185.69                    0.0                100.0   
3     A   Apr    2297      67.56                    0.0                100.0   
4     A   May    1992      58.59                    0.0                100.0   
5     A   Jun    2275      71.09                    0.0                100.0   
6     A   Jul    5334     177.80                    0.0                100.0   
7     A   Aug    3371      99.15                    0.0                100.0   
8     A   Sep    3759     104.42                    0.0                100.0   
9     A   Oct    3529     110.28                    0.0                100.0   

   Appl Percentage Outsource      Cost  Average Cost Per App  \
0                        0.0  727800.0                138.89   
1                        0.0  717350.0                147.06   
2                        0.0  928450.0                156.25   
3                        0.0  337800.0                147.06   
4                        0.0  292950.0                147.06   
5                        0.0  355450.0                156.24   
6                        0.0  889000.0                166.67   
7                        0.0  495750.0                147.06   
8                        0.0  522100.0                138.89   
9                        0.0  551400.0                156.25   

   No Of FTE Processed Appl  
0                    5240.0  
1                    4878.0  
2                    5942.0  
3                    2297.0  
4                    1992.0  
5                    2275.0  
6                    5334.0  
7                    3371.0  
8                    3759.0  
9                    3529.0

In [54]:
#sending data to excel
append_to_excel('D:/StaffOptimizationResults.xlsx', dfbest, 'Best Availability','a')

#### 3.2.2 Percentage of outsourced applications

In [55]:
# write your code here
outsourcedper = round((dfbest['No of Outsourced Appl'].sum() / dfbest['Demand'].sum() ) * 100,2)
print("% of Outsourced Applications are : ", outsourcedper)

% of Outsourced Applications are :  4.11


#### 3.2.3 Average cost per application

In [56]:
# write your code here
avgcost= round(dfbest['Cost'].sum()/dfbest['Demand'].sum()  , 2)
print("Average Cost per Application :", avgcost)

Average Cost per Application : 145.88


# Question 4

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.